# WindLab

Este notebook exemplifica a utilização do pacote WindLab, desenvolvido para facilitar a leitura, análise e visualização de dados de instrumentos LIDAR, como Wind Cube e Zephyr. O WindLab automatiza a manipulação de dados, permitindo realizar cálculos estatísticos, gerar gráficos (como rosa dos ventos) e exportar resultados em formatos como CSV, ideal para relatórios técnicos e publicações científicas.

Neste notebook, apresentaremos exemplos práticos das principais funcionalidades do WindLab.

## Configuração do Ambiente

Esta seção fornece um guia passo a passo para criar um ambiente Conda (ou virtual) para o WindLab, instalar os módulos necessários a partir do requirements.txt e adicionar o ambiente ao Jupyter Notebook.

### Passo 1: Criar o Ambiente Conda

Primeiro, vamos criar um novo ambiente Conda chamado windlab com Python 3.8 ou superior.

In [ ]:
# Criar um novo ambiente Conda chamado 'windlab' com Python 3.10
conda create -n windlab python=3.10

# Ativar o ambiente
conda activate windlab

### Passo 2: Instalar os Pacotes Necessários

Uma vez que o ambiente esteja ativado, instale todas as dependências necessárias a partir do arquivo requirements.txt.

In [ ]:
# Instalar os pacotes necessários
pip install -r requirements.txt

### Passo 3: Adicionar o Ambiente ao Jupyter

Para trabalhar com este ambiente no Jupyter Notebook, precisamos adicioná-lo como um kernel.

In [ ]:
# Instale o pacote ipykernel:

pip install ipykernel

# Adicione o ambiente ao Jupyter:

python -m ipykernel install --user --name=windlab --display-name "Python (windlab)"

### Passo 4: Verificar a Configuração

Para verificar se tudo está configurado corretamente, execute o seguinte comando em uma célula do Jupyter Notebook:

In [2]:
import sys
print(sys.executable)

/home/daniloceano/anaconda3/envs/windlab/bin/python


### Passo 5: Adcionar Caminho do Projeto ao Ambiente

Para a utilização desse notebook, primeiro temos que ajustar o ambiente para que possamos importar os módulos do pacote. Para isso, vamos adicionar o caminho do projeto ao sys.path manualmente:

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

## Leitura e manipulação de dados

O primeiro passo na utilização do programa envolve a leitura de um arquivo bruto de dados e sua manipulação. Para isso, o **WindLab** utiliza as bibliotecas **Pandas** e **Xarray**:

- **Pandas** é responsável por ler o arquivo bruto tabulado do LIDAR.
- **Xarray** transforma os dados em um objeto multidimensional (3D), permitindo uma manipulação mais intuitiva e facilitada, semelhante a trabalhar com arquivos NetCDF.

No exemplo abaixo, vamos abrir um dado de teste criado especialmente para fins de teste dos módulos do projeto. Este refere-se a dados do instrumento de LIDAR WindCube, para todo o ano de 2020, mas selecionando apenas dados a cada 3 horas, para diminuir a quantidade de observações e, assim, agilizar e facilitar a manipulação dos dados.

In [2]:
from windlab import WindDataAccessor

ds = WindDataAccessor.windcube("../data/subset_data_2020_3h.rtd")
print(ds)

ValueError: need at least one array to stack

Podemos ver que o objeto gerado possui as coordenadas **tempo** e **altura**, além de diferentes variáveis, como a **velocidade do vento**, suas componentes nos eixos **X, Y e Z**, e a **direção do vento** (em graus). 

### Importante:
A altura indicada está relativa ao nível do mar mais a altura do aparelho instalado. Por exemplo, se o Wind Cube está instalado a 100m do nível do mar, soma-se a altura do feixe de luz medido à altura de referência. 

Como o metadado do arquivo não especifica a altura de referência do aparelho, essa altura deve ser passada pelo usuário como argumento no momento de construir o dataset. Veja um exemplo abaixo:

In [ ]:
# Criando uma instância do ReadWindCubeAccessor e carregando o arquivo com uma altura de referência de 100 metros
ds_accessor = ReadWindCubeAccessor("./WLS866-104_2024_08_01__00_00_00.rtd")
ds_accessor.load_data(reference_height=100)

# Acessar diretamente o objeto xarray.Dataset e verificar as alturas ajustadas
dataset = ds_accessor.dataset
print(dataset)

### Seleção de tempos e alturas específicas

Uma vez que os dados são carregados no formato `xarray.Dataset`, o **ReadWindCube** oferece métodos fáceis e eficientes para selecionar intervalos de tempo e alturas específicas. Isso pode ser feito utilizando os métodos `.sel` (seleção por coordenadas) e `.isel` (seleção por índices).

#### Método `.sel`: Seleção por coordenadas

O método `.sel` permite selecionar dados com base nos valores exatos das coordenadas, como datas e alturas específicas. Por exemplo, você pode selecionar dados para um período de tempo ou uma altura específica em metros.

**Exemplo: Seleção por intervalo de tempo e altura específica**

In [ ]:
# Selecionando dados entre 2024-08-01 00:00:00 e 2024-08-01 02:00:00 para a altura de 50 metros
subset_accessor = ds_accessor.sel(time=slice('2024-08-01T00:00:00', '2024-08-01T02:00:00'), height=150)
print(subset_accessor.dataset)

Neste exemplo, selecionamos o intervalo de tempo desejado usando a função `slice`, e a altura é escolhida diretamente com o valor exato (neste caso, 150 metros). Isso permite visualizar os dados apenas para aquele intervalo e altura.

#### Método `.isel`: Seleção por índices 

O método `.isel` permite a seleção de dados com base nos índices das coordenadas, ou seja, você pode escolher as posições numéricas das coordenadas de tempo e altura em vez de seus valores exatos.

In [ ]:
# Selecionando os primeiros 100 passos de tempo e o primeiro índice de altura
subset_accessor = ds_accessor.isel(time=slice(0, 100), height=0)
print(subset_accessor.dataset)

Neste exemplo, estamos selecionando os primeiros 100 passos de tempo e o primeiro índice de altura no dataset. O primeiro índice de altura corresponde à menor altura no dataset (por exemplo, se as alturas são 50, 60 e 70 metros, o índice 0 corresponde a 50 metros).

### Cálculo do desvio padrão da velocidade do vento e remoção de tendência

O **ReadWindCube** oferece uma função útil para calcular o desvio padrão da velocidade do vento, após remover tendências dos dados. Essa operação é particularmente útil para identificar a variabilidade do vento em diferentes alturas e durante diferentes intervalos de tempo, à partir de uma série estacionária.

#### Função `compute_std_detrended_data`

Essa função realiza o cálculo do desvio padrão para uma variável (por padrão, a velocidade do vento) em uma altura específica, utilizando uma janela deslizante para suavizar as variações e remover tendências de longo prazo.

#### Parâmetros de entrada:
- **height** (*int*): Altura em metros para a qual o desvio padrão será calculado.
- **variable** (*str*): Nome da variável para a qual o cálculo será realizado (o padrão é 'Wind Speed (m/s)').
- **window_size** (*int*): Tamanho da janela deslizante (em número de passos no tempo) para o cálculo do desvio padrão. O padrão é 600 passos no tempo, mas pode ser ajustado conforme necessário.

#### Exemplo prático:

Abaixo, calculamos o desvio padrão com remoção de tendência para a velocidade do vento a 40 metros de altura e, em seguida, plotamos o resultado ao longo do tempo.

In [ ]:
# Calcular o desvio padrão com remoção de tendência para a velocidade do vento a 140 metros
std_detrended_ws140m = ds_accessor.compute_std_detrended_data(140, 'Wind Speed (m/s)')

# Importar biblioteca de plotagem
import matplotlib.pyplot as plt

# Plotar o desvio padrão ao longo do tempo
plt.figure(figsize=(8,6))
plt.plot(dataset.time.values, std_detrended_ws140m.values, c='#c1121f')
plt.title('Desvio Padrão da Velocidade do Vento com Tendência Removida a 140m', fontsize=14)
plt.xlabel('Tempo', labelpad=10, fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.show()

### Exemplo: Cálculo do desvio padrão com remoção de tendência e plotagem da velocidade do vento a 140 metros

Agora, vamos plotar a série temporal da velocidade do vento real e do desvio padrão calculado para podermos comparar visualmente ambas as séries

In [ ]:
# Calcular o desvio padrão com remoção de tendência para a velocidade do vento a 140 metros
std_detrended_ws140m = ds_accessor.compute_std_detrended_data(140, 'Wind Speed (m/s)')

# Velocidade real a 140 metros
ws140m = dataset["Wind Speed (m/s)"].sel(height=140)
print(ws140m)

# Importar biblioteca de plotagem
import matplotlib.pyplot as plt

# Plotar o desvio padrão ao longo do tempo junto com a velocidade do vento original
plt.figure(figsize=(8,6))
plt.plot(dataset.time.values, ws140m.values, c='#003049', label='Velocidade do Vento a 140m')
plt.plot(dataset.time.values, (std_detrended_ws140m + ws140m.mean()).values, c='#c1121f', label='Velocidade com Desvio Padrão Detrended')
plt.title('Desvio Padrão da Velocidade do Vento com Tendência Removida a 140m', fontsize=14)
plt.xlabel('Tempo', labelpad=10, fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
plt.show()


### Plotagem da velocidade do vento em uma única altura

Para facilitar a visualização da velocidade do vento, o **ReadWindCube** possui uma função embutida chamada `plot_variable`. Com esta função, o usuário pode facilmente gerar gráficos de variáveis, como a velocidade do vento, para um nível específico de altura.

#### Função `plot_variable`

A função `plot_variable` gera um gráfico simples da variável desejada para a altura especificada. Isso é útil quando o usuário quer visualizar rapidamente a evolução da velocidade do vento ao longo do tempo para uma única altura, sem a necessidade de realizar operações adicionais.

#### Parâmetros de entrada:
- **height** (*int*): A altura em metros para a qual a variável será plotada.
- **variable** (*str*): A variável a ser plotada (por padrão, 'Wind Speed (m/s)').

#### Exemplo prático:

Neste exemplo, vamos visualizar: velocidade do vento, velocidade do vento na direção X e direção do vento, a uma altura de 140 metros. Além da plotagem básica, o usuário pode personalizar o gráfico gerado através do objeto `ax`, que representa o eixo do gráfico. Abaixo estão algumas customizações possíveis:

In [ ]:
# Velocidade do vento
ax = ds_accessor.plot_variable(140, 'Wind Speed (m/s)')

# Componente X
ax = ds_accessor.plot_variable(140, 'X-wind (m/s)')
ax.tick_params(axis='x', labelrotation=45)  # Rotaciona os rótulos do eixo X

# Direção do vento
ax = ds_accessor.plot_variable(140, 'Wind Direction (°)')
ax.tick_params(axis='x', labelrotation=45)
ax.grid(True)  # Ativa o grid no gráfico
line = ax.lines[0] # Capturando a linha gerada
line.set_color('red')  # Define a cor da linha para vermelho

### Obter dados de velocidade e direção do vento em formato de DataFrame

O **ReadWindCube** oferece uma função para extrair os dados de velocidade e direção do vento para uma altura específica e convertê-los diretamente em um `pandas.DataFrame`. Isso é útil quando se deseja trabalhar com os dados em um formato mais tabular, por exemplo, para fazer cálculos adicionais ou exportar os dados para um arquivo CSV.

#### Função `get_wind_df`

A função `get_wind_df` retorna um `DataFrame` com as colunas 'Wind Speed (m/s)' e 'Wind Direction (°)' para a altura especificada.

#### Parâmetros de entrada:
- **height** (*int*): A altura em metros para a qual os dados serão extraídos.

#### Exemplo prático:

Neste exemplo, vamos obter a velocidade e a direção do vento a 40 metros e visualizar os primeiros dados retornados:

In [ ]:
# Obter um DataFrame com velocidade e direção do vento a 40 metros
wind_df = ds_accessor.get_wind_df(140)

# Exibir o DataFrame
print(wind_df.head())

### Visualização de Rosas dos Ventos

Uma funcionalidade bastante útil do ReadWindCube é a possibilidade de gerar rosas dos ventos a partir dos dados de velocidade e direção do vento em diferentes alturas. A função `plot_wind_rose` permite que você visualize uma rosa dos ventos para uma altura específica, com a possibilidade de calcular médias em janelas de tempo específicas.

#### Parâmetros da Função `plot_wind_rose` 

* **`height`**: Altura (em metros) para a qual será gerada a rosa dos ventos.
* **`averaging_window`**: (Opcional) Janela de tempo para calcular a média dos dados. O formato segue as regras do Pandas, como `'1h'` para média horária, `'10T'` para média a cada 10 minutos, etc.
* **`colormap`**: (Opcional) O mapa de cores utilizado para a plotagem. O valor padrão é `'viridis'`, mas você pode escolher diferentes colormaps como `'coolwarm'`, `'plasma'`, etc.
* **`period`**: (Opcional) Um período para filtrar os dados. Pode ser um mês específico (por exemplo, `'January'`, `'February'`) ou uma estação do ano (usando os códigos meteorológicos `'DJF'` para verão austral, `'JJA'` para inverno austral, etc.).
* **Retorno**: A função retorna o objeto `WindroseAxes` para que você possa modificar ou salvar o gráfico posteriormente.

#### Exemplo 1: Plotar a Rosa dos Ventos sem Média

No exemplo abaixo, plotamos a rosa dos ventos para a altura de 140 metros sem realizar médias temporais.

In [ ]:
# Plotar a rosa dos ventos para a altura de 140 metros sem média temporal
ds_accessor.plot_wind_rose(140)
plt.show()

#### Exemplo 2: Plotar a Rosa dos Ventos com Média Temporal 

A função `plot_wind_rose` também permite calcular médias para diferentes janelas de tempo. Abaixo, mostramos como calcular a média de velocidade e direção do vento a cada 1 hora e, em seguida, plotar a rosa dos ventos para a altura de 40 metros. Além disso, você pode modificar o título e salvar a figura posteriormente.

In [ ]:
# Plotar a rosa dos ventos para a altura de 140 metros com média horária e um mapa de cores diferente
ax = ds_accessor.plot_wind_rose(140, averaging_window='1h', colormap='coolwarm')
ax.set_title("Rosa dos Ventos a 40m - Média Horária", fontsize=16)
plt.show()

# Salvar a figura
ax.figure.savefig('rosa_dos_ventos_140m_horario.png')

### Geração de Tabela de Distribuição de Vento

A função `generate_wind_distribution_table` tem como objetivo gerar uma tabela de distribuição cumulativa ou binned de vento. Cada linha pode representar a frequência cumulativa para velocidades de vento abaixo de um determinado limite ou a frequência dentro de intervalos de velocidade, enquanto cada coluna representa direções de vento agrupadas em bins de ±15° ao redor de um valor central (por exemplo, 30°, 60°, etc.).

Essa função é útil para análise de frequência de vento em diferentes direções e velocidades e pode ser usada para gerar tabelas que resumem as características do vento em diferentes alturas.

#### Parâmetros:
- **height (int)**: A altura em metros na qual calcular a distribuição de vento.
- **speed_thresholds (list, opcional)**: Lista com os limites de velocidade do vento (em m/s). O padrão é [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32].
- **direction_bins (list, opcional)**: Lista com os limites de bins de direção do vento (em graus). O padrão é [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360].
- **period (str, opcional)**: Um parâmetro opcional para filtrar dados de uma estação meteorológica específica ou mês. As estações podem ser representadas pelas siglas "DJF" (Verão), "MAM" (Outono), "JJA" (Inverno), e "SON" (Primavera). Além disso, também pode ser especificado um mês, como "January" (Janeiro), "February" (Fevereiro), etc.
- **mode (str, opcional)**: O modo de cálculo. Pode ser "accumulate" para probabilidades cumulativas ou "bins" para probabilidades distribuídas em intervalos. O padrão é "accumulate".

#### Retorno:
- **pd.DataFrame**: Um DataFrame contendo a tabela de distribuição de vento, onde as linhas são os limites de velocidade do vento (cumulativos ou intervalos) e as colunas são os bins de direção do vento, além de uma coluna "Omni" para a soma das frequências em todas as direções.

In [ ]:
# Geração da tabela de distribuição de vento a 140 metros de altura
wind_distribution_table = ds_accessor.generate_wind_distribution_table(140)

# Exibição da tabela gerada
print(wind_distribution_table)

O arquivo que estávamos usando para teste possui, na verdade, dados apenas para as primeiras 10 horas do dia 08 de agosto de 2024. Por conta disso, a tabela de distribuição gerada anteriormente não apresentou muitos valores, o que pode dificultar uma análise mais abrangente da funcionalidade. Para podermos ter uma visão mais clara e completa do potencial dessa função, utilizaremos agora um arquivo de teste criado especificamente para este guia. Este novo arquivo contém dados fictícios para o ano de 2023, com velocidades de vento geradas a partir de uma distribuição Weibull aleatória, e uma resolução temporal de 1 hora. Com esse novo arquivo, iremos criar um novo objeto `ReadWindCubeAccessor` e construir a tabela de distribuição de vento para análise.

In [ ]:
# Criando o novo objeto ReadWindCubeAccessor com o arquivo de teste fictício
ds_accessor = ReadWindCubeAccessor("./dummy_data_2023.rtd")

# Carregar os dados do arquivo de teste
ds_accessor.load_data(reference_height=100)

# Gerar a tabela de distribuição de vento a 140 metros de altura para o ano de 2023
wind_distribution_table_test = ds_accessor.generate_wind_distribution_table(140)

# Exibir a tabela gerada
print(wind_distribution_table_test)

#### Exemplo de Uso: Filtrando por Mês ou Estação 
Além de gerar a tabela de distribuição para o conjunto de dados completo, podemos também filtrar por um período específico, como um mês ou uma estação do ano. 

##### Exemplo: Filtrando por Mês

In [ ]:
# Gerar a tabela de distribuição de vento para o mês de Janeiro
wind_distribution_january = ds_accessor.generate_wind_distribution_table(140, period='January')
print(wind_distribution_january)

##### Exemplo: Filtrando por Estação (Verão)

In [ ]:
# Gerar a tabela de distribuição de vento para o verão (Dezembro, Janeiro, Fevereiro)
wind_distribution_summer = ds_accessor.generate_wind_distribution_table(140, period='DJF')
print(wind_distribution_summer)

Também é possível fazer o gráfico da rosa dos ventos para um mês ou estação específico:

In [ ]:
# Plotar a rosa dos ventos para 140 metros no mês de janeiro
ax = ds_accessor.plot_wind_rose(140, colormap='coolwarm', period='January')
plt.show()

#### Exemplo de Uso: Usando bins ao invés de valores acumulados

Nesse caso, o uso do argumento 'mode' em conjunto com a opção 'bins' faz com que cada linha represente a frequência relativa de ocorrência de valores entre os limites determinados na primeira coluna, sendo que o valor da esquerda é incluído, enquanto que os valores têm que ser menores que o valor à direita. 

Por exemplo, se o intervalo for `2-3`, isso significa que estamos considerando a frequência de ocorrência de velocidades de vento maiores ou iguais a 2 m/s e menores que 3 m/s. Cada coluna da tabela representa um bin de direção do vento, com cada valor indicando a porcentagem de ocorrência relativa àquela direção específica dentro do intervalo de velocidade.

A tabela gerada com o modo 'bins' é útil para entender a distribuição do vento em diferentes faixas de velocidade e direções, permitindo uma análise detalhada dos padrões de vento em uma determinada altura.

In [ ]:
# Definir opções para mostrar todas as linhas e colunas
wind_distribution_bins = ds_accessor.generate_wind_distribution_table(140, mode='bins')
print(wind_distribution_bins)

### Geração de Tabela de Cobertura de Dados

A função `generate_data_coverage_table` gera uma tabela de cobertura de dados que verifica a quantidade de dados faltantes ao longo do tempo. Cada linha da tabela representa um mês, enquanto cada coluna representa um dia do mês. A célula correspondente mostra a porcentagem de cobertura de dados disponível para aquela combinação de mês e dia.

Além disso, a função possui um argumento opcional `plot` que, quando definido como `True`, gera um gráfico visual da cobertura de dados, semelhante ao apresentado na imagem anterior. Esse gráfico utiliza um heatmap para mostrar visualmente a presença (ou ausência) de dados ao longo do tempo, facilitando a análise rápida de lacunas na coleta de dados.

Essa funcionalidade é extremamente útil para verificar a qualidade dos dados antes de realizar qualquer análise mais detalhada, garantindo que não existam grandes períodos com falta de dados.

In [ ]:
coverage_table = ds_accessor.generate_data_coverage_table(140, plot=True)
print(coverage_table)

### Geração de Tabela de Velocidade Média do Vento por Hora e Mês

A função `generate_average_wind_speed_table` gera uma tabela contendo os valores médios da velocidade do vento para cada hora do dia, separados por mês, além de agrupamentos sazonais (verão, outono, inverno e primavera) e uma média global. Cada linha da tabela representa uma hora do dia, enquanto cada coluna representa um mês ou um agrupamento sazonal.

A última linha da tabela representa a média mensal de velocidade do vento para cada coluna. A função também possui um argumento opcional `plot`, que, quando definido como `True`, cria um gráfico visual da tabela. As cores do gráfico representam a razão entre a velocidade média do vento e a velocidade máxima, facilitando a interpretação dos padrões sazonais e diários da velocidade do vento.

Essa funcionalidade é útil para identificar padrões de vento ao longo do dia e das estações do ano, proporcionando uma visão detalhada dos comportamentos de vento em diferentes períodos.



In [ ]:
average_wind_speed_table = ds_accessor.generate_average_wind_speed_table(140, plot=True)
print(average_wind_speed_table)
plt.show()